In [ ]:
import numpy as np

def compute_embeddings(model, generator):
    embeddings = []
    labels = []
    for i in range(len(generator)):
        batch_x, _ = generator[i]
        anchor_embeddings, positive_embeddings, negative_embeddings = model.predict_on_batch(batch_x)
        embeddings.append(np.concatenate([anchor_embeddings, positive_embeddings, negative_embeddings], axis=0))
        labels.extend(generator.labels[generator.indices[i * generator.batch_size: (i + 1) * generator.batch_size]])
    embeddings = np.vstack(embeddings)
    labels = np.array(labels)
    return embeddings, labels

# Compute embeddings for the training set
train_embeddings, train_labels = compute_embeddings(model, train_triplet_gen)

# For evaluation, you would typically compare these embeddings using a metric like cosine similarity or Euclidean distance.
# For simplicity, here we'll just print the shape of the embeddings.
print('Train embeddings shape:', train_embeddings.shape)
print('Train labels shape:', train_labels.shape)


In [1]:
from tensorflow.keras import backend as K
import gc

K.clear_session()
gc.collect()

0

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = tf.keras.applications.InceptionResNetV2(
            input_shape=(299,299,3),
            include_top=False,
            weights='imagenet'
        )
        

inputs = Input(shape=(299,299,3))
x = base_model(inputs, training=False)  # Keep BatchNorm layers in inference mode
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
outputs = Dense(7, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 inception_resnet_v2 (Functi  (None, 8, 8, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 7)                 3591      
                                                                 
Total params: 55,127,271
Trainable params: 55,066,727
Non-tra

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

class InceptionResNet_model:
    def __init__(self, image_size):
        self.image_shape = (image_size, image_size, 3)

    def configure_model(self, freeze_base_model=True):
        base_model = tf.keras.applications.InceptionResNetV2(
            input_shape=self.image_shape,
            include_top=False,
            weights='imagenet'
        )
        
        # Initially freeze the base model
        if freeze_base_model:
            base_model.trainable = False

        inputs = Input(shape=self.image_shape)
        x = base_model(inputs, training=False)  # Keep BatchNorm layers in inference mode
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation='relu')(x)
        outputs = Dense(7, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs)
        
        return model

    def compile_and_train(self, model, train_data, val_data, initial_epochs=10, fine_tune_epochs=10):
        # Compile the model for the initial training
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Train the top layers with the base model frozen
        model.fit(train_data, validation_data=val_data, epochs=initial_epochs)
        
        # Unfreeze the base model for fine-tuning
        model.layers[1].trainable = True
        
        # Re-compile the model with a lower learning rate
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
                      loss='categorical_crossentropy', 
                      metrics=['accuracy'])
        
        # Fine-tune the entire model
        model.fit(train_data, validation_data=val_data, epochs=fine_tune_epochs)
        
        return model

# Example usage:
image_size = 224
model_instance = InceptionResNet_model(image_size)
model = model_instance.configure_model()

# Assuming train_data and val_data are your training and validation data generators
train_data = ...  # Define your training data generator
val_data = ...  # Define your validation data generator

# Compile and train the model
trained_model = model_instance.compile_and_train(model, train_data, val_data, initial_epochs=10, fine_tune_epochs=10)
